In [1]:
# opencv-python
import cv2
# mediapipe人工智能工具包
import mediapipe as mp
# 进度条库
from tqdm import tqdm
# 时间库
import time
# 导入python绘图matplotlib
import matplotlib.pyplot as plt
# 使用ipython的魔法方法，将绘制出的图像直接嵌入在notebook单元格中
%matplotlib inline
import numpy as np

In [2]:
# 导入solution
mp_pose = mp.solutions.pose
# #导入绘图函数
mp_drawing = mp.solutions.drawing_utils

# 导入模型
pose = mp_pose.Pose(static_image_mode=False,
                    # 是静态图片还是连续视频帧
                    model_complexity=2,
                    # 选择人体姿态关键点检测模型，0性能差但快，2性能好但慢，1介于两者之间
                    smooth_landmarks=True,
                    # 是否平滑关键点
                    enable_segmentation=False,
                    # 是否人体抠图
                    min_detection_confidence=0.5,  # 置信度阈值
                    min_tracking_confidence=0.5)  # 追踪阈值

In [3]:
def process_frame(img):
    # 记录该帧开始处理的时间
    start_time = time.time()
    # 获取图像宽高
    h, w = img.shape[0], img.shape[1]

    # BGR转RGB
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     print("True0")
    # 将RGB图像输入模型，获取预测结 果
    results = pose.process(img_RGB)
    print("True")
    if results.pose_landmarks:  # 若检测出人体关键点
        print("True1")
        # 可视化关键点及骨架连线
        mp_drawing.draw_landmarks(
            img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        for i in range(33):  # 遍历33个关键点
            # 获取该关键点的三维坐标
            print("True2")
            cx = int(results.pose_landmarks.landmark[i].x*w)
            cy = int(results.pose_landmarks.landmark[i].y*h)
            cz = results.pose_landmarks.landmark[i].z
            
            radius = 5

            if i == 0:  # 鼻尖
                img = cv2.circle(img, (cx, cy), radius, (0, 0, 255), -1)
            elif i in [11, 12]:  # 肩膀
                img = cv2.circle(img, (cx, cy), radius, (223, 155, 6), -1)
            elif i in [23, 24]:  # 髋关节
                img = cv2.circle(img, (cx, cy), radius, (1, 240, 255), -1)
            elif i in [13, 14]:  # 胳膊肘
                img = cv2.circle(img, (cx, cy), radius, (140, 47, 240), -1)
            elif i in [25, 26]:  # 膝盖
                img = cv2 .circle(img, (cx, cy), radius, (0, 0, 255), -1)
            elif i in [15, 16, 27, 28]:  # 手腕和脚腕
                img = cv2.circle(img, (cx, cy), radius, (223, 155, 60), -1)
            elif i in [17, 19, 21]:  # 左手
                img = cv2.circle(img, (cx, cy), radius, (94, 218, 121), -1)
            elif i in [18, 20, 22]:  # 右手
                img = cv2.circle(img, (cx, cy), radius, (16, 144, 247), -1)
            elif i in [27, 29, 31]:  # 左脚
                img = cv2.circle(img, (cx, cy), radius, (29, 123, 243), -1)
            elif i in [28, 30, 32]:  # 右脚
                img = cv2.circle(img, (cx, cy), radius, (193, 182, 255), -1)
            elif i in [9, 10]:  # 嘴
                img = cv2.circle(img, (cx, cy), radius, (205, 235, 255), -1)
            elif i in [1, 2, 3, 4, 5, 6, 7, 8]:  # 眼及脸颊
                img = cv2.circle(img, (cx, cy), radius, (94, 218, 121), -1)
            else:  # 其它关键点
                img = cv2.circle(img, (cx, cy), radius, (0, 255, 0), -1)
            
    else:
        scaler = 1
        failure_str = 'No Person'
        img = cv2.putText(img, failure_str, (25*scaler, 100 * scaler),
                          cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255))
        print( '从图像中未检测出人体关键点，报错。')
        
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)
    
    scaler = 1
    # 在图像上写FPS数值， 参数依次为:图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
    img = cv2.putText(img, 'FPS:'+str(int(FPS)), (25 * scaler, 50 * scaler),
                      cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (0, 255, 0))
    return img

### 视频逐帧处理(模板)

In [4]:
def generate_video(input_path="E:/opencv/opencv-Advanced courses/Chapter 2 – Pose Estimation/1.mp4"): 
    filehead = input_path.split('/')[-1]
    output_path = "out-" + filehead
    
    print( '视频开始处理' ,input_path)
    
    #获取视频总帧数
    cap=cv2.VideoCapture(input_path)
    frame_count = 0
    
    while (cap.isOpened()):
        success, frame = cap. read()
        frame_count += 1
        if not success:
            break
    cap.release()
    
    print("视频总帧数", frame_count)
    
    # cv2.namedWindow( 'Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH),cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # fourcc = int(cap. get (Cv2.CAP_ PROP_ FOURCC))
    # fourcc = cv2. videoWriter fourcc(* 'XVID')
    fourcc = cv2.VideoWriter_fourcc(* 'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    out = cv2.VideoWriter(output_path, fourcc,fps, (int(frame_size[0]), int(frame_size[1])))
    
    #进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
#                 print(success)  # True
                if not success:
                    break
                    
                #处理帧
                # frame path = './temp_ frame. png'
                # cv2. imwrite(frame_ path, frame)
#                 try:
                # print("cap.isOpened()")  # True
                frame = process_frame(frame)
                print("True")
#                 except:
#                     print('error')
#                     pass
                if success == True: 
                    # cv2. imshow( 'Video Processing', frame)
                    out.write(frame)
                    
                    #进度条更新-帧
                    pbar.update(1)
                    
                # if cv2.waitKey(1) & 0xFF == ord('q'):
                    # break
                
        except:
            print('中途中断')
            pass
        
    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print( '视频已保存', output_path)

In [ ]:
generate_video(input_path="E:/opencv/opencv-Advanced courses/Chapter 2 – Pose Estimation/1.mp4")

视频开始处理 E:/opencv/opencv-Advanced courses/Chapter 2 – Pose Estimation/1.mp4
